In [16]:
import numpy as np
from scipy.stats import norm
from scipy.optimize import fsolve

### **1) QED Model**

In [17]:
#Hazard Function for standard normal distribution

def hazard_function_normal(x, mean=0, std_dev=1):
    pdf = norm.pdf(x, loc=mean, scale=std_dev)
    cdf = norm.cdf(x, loc=mean, scale=std_dev)
    if cdf == 1:
        return 0
    return pdf / (1 - cdf)

In [18]:
# Function to define equation whose roots will give us beta*

def equation_to_solve(beta, mu, M, lamda):
    term1 = hazard_function_normal(beta * np.sqrt(mu)) - beta * np.sqrt(mu)
    term2 = 1 + ((1 / np.sqrt(mu)) * (hazard_function_normal(beta * np.sqrt(mu)) / hazard_function_normal(-beta)))
    lhs = M * np.sqrt(lamda)
    return term1 / term2 - lhs

In [19]:
# Constant_Inputs
mu = 1/13         #service rate
M = 0.1         #acceptable abandonment probability
lamda = 32      #arrival rate
g0 = 1          #patience density at origin (?)

# Initial guess for beta
beta_initial_guess = 0.5

In [20]:
#Solve for beta*

beta_solution = fsolve(equation_to_solve, beta_initial_guess, args=(mu, M, lamda))

In [21]:
# Function to calculate number of servers (QED Model)
def optimal_n(beta, mu, lamda):
    return np.ceil(((beta * np.sqrt(lamda) / np.sqrt(mu)) + (lamda / mu)))

In [22]:
n_solution = optimal_n(beta_solution, mu, lamda)

In [23]:
#Calculate probability of abandonment

term1 = hazard_function_normal(beta_solution * np.sqrt(mu)) - beta_solution * np.sqrt(mu)
term2 = 1 + ((1 / np.sqrt(mu)) * (hazard_function_normal(beta_solution * np.sqrt(mu)) / hazard_function_normal(-beta_solution)))
p_abandonment = g0*(term1 / term2) / lamda

In [24]:
print("Given Inputs:")
print(f"lamda: {lamda}")
print(f"mu: {mu}")
print(f"M: {M}")
print("Given Outputs:")
print(f"Optimal beta: {beta_solution}")
print(f"Optimal SERVERS: {n_solution}")
print(f"Probability of abandonment: {p_abandonment}")

Given Inputs:
lamda: 32
mu: 0.07692307692307693
M: 0.1
Given Outputs:
Optimal beta: [-1.86681251]
Optimal SERVERS: [378.]
Probability of abandonment: [0.01767767]


### **2) QED+ED Model**

## Define alpha, and T

1. Assuming that patience function is a normal distribution with mu=3, sigma=2

2. alpha = 0.1 (Max possible abandonment)

3. T = 1.5 (Limit for abandonment time)

In [25]:
alpha = 0.1
T = 0.5
mean = 3
sigma = 1
lamda = 32
mu = 1/13

In [26]:
def delta_star(mean, sigma, T, mu, alpha):
    g_pdf = norm.pdf(T, loc=mean, scale=sigma)
    G_cdf = norm.cdf(T, loc=mean, scale=sigma)
    if G_cdf == 0:
        return 0
    inv_cdf = norm.ppf(alpha / 1 - G_cdf)
    delta = inv_cdf * np.sqrt(g_pdf / mu)
    return delta

In [27]:
delta = delta_star(mean, sigma, T, mu, alpha)
print(f"The value of delta is: {delta}")

The value of delta is: -0.6290442280765589


In [28]:
G_cdf = norm.cdf(T, loc=mean, scale=sigma)
n_star_ED_QED = ((1-G_cdf) * (lamda/mu)) + (delta * np.sqrt(lamda/mu))
print(f"The value of n* ED+QED is: {n_star_ED_QED}")

The value of n* ED+QED is: 400.586744048976
